In [51]:
import os
import tensorflow as tf
tf.enable_eager_execution()
from pyson.utils import read_json, multi_thread
import json 
from glob import glob
from simple_waymo_open_dataset_reader import WaymoDataFileReader
import cv2
from tqdm import tqdm
paths = glob('/ssd6/coco_style_1.2/annotations/output_json/*.json')
path_sample_coco_annotation = '/ssd6/coco_style_1.2/annotations/val.json'
from simple_waymo_open_dataset_reader import dataset_pb2, label_pb2, utils
from utils import f_datapath
print(len(paths))

sample = json.load(open(path_sample_coco_annotation))

sample['info'] = {'description': '2D waymo',
 'url': 'waymo.com',
 'version': '1.0',
 'year': 2020,
 'contributor': 'Hai Anh',
 'date_created': '2019/04/06'}

def image_to_coco_dict(image_path, index):
    h, w = cv2.imread(image_path).shape[:2]
    return {'license': 4,
             'file_name': os.path.basename(image_path),
             'height': h,
             'width': w,
             'id': index,
    }
anno_id = 0
def annotation_to_dict(value, image_index):
    global anno_id
    rt = []
    box_3d_list = value['box_3d_list']
    counts = value['counts']
    cates = value['cates']
    box_2d_list = value['box_2d_list']
    for count, box_3d, (cate, box_id) in zip(counts, box_3d_list, cates):
        if box_3d is not None:
            ann = {'image_id': image_index,
                         'category_id': cate,
                         'box_3d': box_3d,
                         'iscrowd': -1,
                         'count':count,
                         'id': anno_id,
                         'box_id':box_id,
                  }
            anno_id += 1
            rt.append(ann)
    for a,b, cate, box_id in box_2d_list:
        ann = {'image_id': image_index,
                             'category_id': cate,
                             'box': [*a, *b],
                             'iscrowd': -1,
                             'id': anno_id,
                             'box_id':box_id,
                      }
        anno_id += 1
        rt.append(ann)
    return rt



# get label 3d
def f_frame_data(frame_data):
    output_dir = '/ssd6/coco_style_1.2/'
    frame, frame_id, frame_name = frame_data
    result = {}
    for camera_name in range(1,6,1):
        camera = utils.get(frame.images, camera_name)
        image_name = dataset_pb2.CameraName.Name.Name(camera_name)
        output_name = os.path.join(output_dir, 'images', f'{frame_name}_{frame_id}_{image_name}.jpg')
        result[output_name] = [(l.type, l.id) for l in frame.laser_labels]
    return result


def _tf_path_from_path(path):
    tf_path = os.path.basename(path).split('.tfrecord')[0]+".tfrecord"
    tf_path = os.path.join('/toyota/waymo/training_1.2', tf_path)
    tf_path = os.path.basename(path).split('.tfrecord')[0]+".tfrecord"

        
    if not os.path.exists(tf_path):
        tf_path = os.path.join('/toyota/waymo/training_1.2', tf_path)
    else:
        tf_path = os.path.join('/ssd6/waymo/tfrecord_train', tf_path)
    return tf_path
def get_3d_label(path):
    tf_path = _tf_path_from_path(path)
    frames_data = f_datapath(tf_path)
    label_3d = multi_thread(f_frame_data, frames_data)
    labels_3d = label_3d[0]
    for _ in label_3d: labels_3d.update(_)
    return labels_3d

images = []
annotations = []
image_id = 0
for path in tqdm(paths):
    x = read_json(path)
    labels_3d = get_3d_label(path)
    for _, (key, value) in enumerate(x.items()):
        value['cates'] = labels_3d[key]
        images.append(image_to_coco_dict(key, image_id))
        annos = annotation_to_dict(value, image_id)
        for anno in annos:
            annotations.append(anno)
        image_id+=1
